In [201]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lag, date_format, concat_ws, to_timestamp, to_date, expr
from pyspark.sql.window import Window

spark = SparkSession.builder.appName("FB").getOrCreate()

df = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0.csv", header=True, inferSchema=True)
df2 = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0 (1).csv", header=True, inferSchema=True)
df3 = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0 (2).csv", header=True, inferSchema=True)
df4 = spark.read.csv("C:/Users/ThinkPad/GitProjects/FootballPredictions/E0 (3).csv", header=True, inferSchema=True)


In [202]:
df.show(5)

+---+----------+-------------------+-----------+-------------+----+----+---+----+----+---+---------+---+---+---+---+---+---+---+---+---+---+---+---+-----+-----+-----+----+----+----+----+----+----+----+----+-----+----+---+----+----+----+----+----+----+----+----+----+-----+--------+--------+-----+-----+-------+-------+-------+-------+-----+-------+-------+----+----+------+------+------+------+------+------+------+----+----+----+----+----+----+----+----+-----+----+----+----+----+----+----+-----+-----+-----+-----+-----+-----+---------+---------+------+------+--------+--------+--------+--------+-----+--------+--------+-----+-----+-------+-------+-------+-------+
|Div|      Date|               Time|   HomeTeam|     AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|  Referee| HS| AS|HST|AST| HF| AF| HC| AC| HY| AY| HR| AR|B365H|B365D|B365A| BWH| BWD| BWA| IWH| IWD| IWA| PSH| PSD|  PSA| WHH|WHD| WHA| VCH| VCD| VCA|MaxH|MaxD|MaxA|AvgH|AvgD| AvgA|B365>2.5|B365<2.5|P>2.5|P<2.5|Max>2.5|Max<2.5|Avg>2.5|Avg<2.5|

In [203]:
df = df.union(df2)
df = df.union(df3)
df = df.union(df4)


In [204]:
df = df.drop("Div")
df = df.select("Date", "Time", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "Referee", "HS", "AS", "HST", "AST", "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR")
df.groupBy("Referee").count().orderBy("count", ascending=False).show(300)

+------------+-----+
|     Referee|count|
+------------+-----+
|    A Taylor|  113|
|    M Oliver|  108|
|   P Tierney|  103|
|    C Pawson|   89|
|   S Attwell|   85|
|    S Hooper|   81|
|    A Madley|   79|
|     D Coote|   78|
|  C Kavanagh|   73|
|     R Jones|   69|
|    P Bankes|   59|
|  A Marriner|   57|
|   D England|   56|
|  M Atkinson|   52|
|      M Dean|   51|
|      J Moss|   49|
|   J Gillett|   46|
|    J Brooks|   45|
|    K Friend|   43|
|     G Scott|   37|
| M Salisbury|   32|
|  T Robinson|   23|
|   T Bramall|   19|
|T Harrington|   19|
|   S Barrott|   15|
|     L Mason|   11|
|      D Bond|    9|
|     J Smith|    5|
|   S Allison|    3|
|    R Madley|    2|
|     R Welch|    2|
|     L Smith|    2|
|   M Donohue|    2|
|  J Gillett |    1|
|     S Singh|    1|
|      A Moss|    1|
+------------+-----+



In [205]:
df = df.withColumn("corrected_date", expr("concat('20', substr(date, 9, 2), '-', substr(date, 4, 2), '-', substr(date, 1, 2))"))
df = df.withColumn("corrected_date_type", to_date(col("corrected_date")))
df = df.withColumn("updated_datetime", (to_timestamp(concat_ws(' ', date_format(col("corrected_date_type"), 'yyyy-MM-dd'), date_format(col("Time"), 'HH:mm:ss')))))
df = df.drop("Date", "Time", "corrected_date")
df = df.withColumnRenamed("corrected_date_type", "Date")
df = df.withColumnRenamed("updated_datetime", "Time")
df.show()
#df.printSchema()

+----------------+----------------+----+----+---+----+----+---+---------+---+---+---+---+---+---+---+---+---+---+---+---+----------+-------------------+
|        HomeTeam|        AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|  Referee| HS| AS|HST|AST| HF| AF| HC| AC| HY| AY| HR| AR|      Date|               Time|
+----------------+----------------+----+----+---+----+----+---+---------+---+---+---+---+---+---+---+---+---+---+---+---+----------+-------------------+
|         Burnley|        Man City|   0|   3|  A|   0|   2|  A| C Pawson|  6| 17|  1|  8| 11|  8|  6|  5|  0|  0|  1|  0|2023-08-11|2023-08-11 20:00:00|
|         Arsenal|   Nott'm Forest|   2|   1|  H|   2|   0|  H| M Oliver| 15|  6|  7|  2| 12| 12|  8|  3|  2|  2|  0|  0|2023-08-12|2023-08-12 12:30:00|
|     Bournemouth|        West Ham|   1|   1|  D|   0|   0|  D| P Bankes| 14| 16|  5|  3|  9| 14| 10|  4|  1|  4|  0|  0|2023-08-12|2023-08-12 15:00:00|
|        Brighton|           Luton|   4|   1|  H|   1|   0|  H|  D Coote| 27|  9| 

In [206]:
#Last 5 game results

window_spec = Window.partitionBy("HomeTeam").orderBy("Time")

df_with_last_result = df.withColumn("last_result", lag("FTR", 5).over(window_spec))

df_with_last_result.show()

+--------+----------------+----+----+---+----+----+---+----------+---+---+---+---+---+---+---+---+---+---+---+---+----------+-------------------+-----------+
|HomeTeam|        AwayTeam|FTHG|FTAG|FTR|HTHG|HTAG|HTR|   Referee| HS| AS|HST|AST| HF| AF| HC| AC| HY| AY| HR| AR|      Date|               Time|last_result|
+--------+----------------+----+----+---+----+----+---+----------+---+---+---+---+---+---+---+---+---+---+---+---+----------+-------------------+-----------+
| Arsenal|        West Ham|   2|   1|  H|   1|   1|  D|  M Oliver|  7| 14|  3|  3| 11| 13|  7|  5|  0|  1|  0|  0|2020-09-19|2020-09-19 20:00:00|       NULL|
| Arsenal|Sheffield United|   2|   1|  H|   0|   0|  D|   L Mason|  6|  6|  5|  2|  3|  9|  2|  4|  0|  1|  0|  0|2020-10-04|2020-10-04 14:00:00|       NULL|
| Arsenal|       Leicester|   0|   1|  A|   0|   0|  D|  C Pawson| 12|  6|  4|  2| 13|  9|  9|  3|  3|  5|  0|  0|2020-10-25|2020-10-25 19:15:00|       NULL|
| Arsenal|     Aston Villa|   0|   3|  A|   0|   1| 